In [1]:
import numpy as np
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm

In [2]:
# Gene expression data
exp_data = pd.read_csv("GD462.GeneQuantRPKM.50FN.samplename.resk10.txt", sep="\t", index_col=[0,1,2,3])
exp_data

,,,,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,HG00105,HG00106,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
TargetID,Gene_Symbol,Chr,Coord,,,,,,,,,,,,,,,,,,,,,
ENSG00000152931.6,ENSG00000152931.6,5,59783540,0.101858,0.078110,0.048981,0.118597,0.004035,0.010925,-0.000901,-0.006706,0.098863,0.045285,...,0.088601,0.240010,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
ENSG00000183696.9,ENSG00000183696.9,7,48128225,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,7.348876,8.180940,8.721889,8.169477,...,13.428205,6.094500,12.536000,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
ENSG00000139269.2,ENSG00000139269.2,12,57846106,1.199910,1.573572,0.521616,1.447225,3.565791,1.982681,0.675305,3.817395,2.561376,1.231049,...,3.225880,1.996067,2.854923,2.267343,1.331201,2.187895,1.004250,3.003316,1.984362,1.684954
ENSG00000169129.8,ENSG00000169129.8,10,116164515,0.831940,0.069778,0.931086,0.620941,1.660668,0.570481,1.259393,0.734784,1.479124,1.548653,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
ENSG00000134602.11,ENSG00000134602.11,X,131157293,27.646422,24.395572,16.445374,24.806650,25.113349,19.233988,27.881116,27.194117,28.579857,27.226416,...,25.079490,28.725528,24.450520,27.264069,26.912814,29.509210,26.462331,25.624009,25.707741,22.824957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235472.1,ENSG00000235472.1,13,29172970,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,35.843048,40.455377,32.287466,37.636318,...,47.905182,33.224977,39.827675,27.096811,34.686342,37.386766,30.598840,33.516674,32.017940,38.341888
ENSG00000114423.14,ENSG00000114423.14,3,105588396,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,15.690696,11.798242,15.461657,18.126433,...,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117
ENSG00000243312.2,ENSG00000243312.2,4,87791344,1.112114,0.831797,0.253228,0.271568,0.486086,1.362640,1.543528,0.535700,0.900239,0.929809,...,1.168991,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.857330,0.703369


In [19]:
exp_data_reset = exp_data.reset_index()
exp_data_reset = exp_data_reset.set_index('Gene_Symbol')
columns_to_drop = ['TargetID', 'Chr', 'Coord']
exp_data_reset = exp_data_reset.drop(columns=columns_to_drop)
exp_data_reset

,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,HG00105,HG00106,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
Gene_Symbol,,,,,,,,,,,,,,,,,,,,,
ENSG00000152931.6,0.101858,0.078110,0.048981,0.118597,0.004035,0.010925,-0.000901,-0.006706,0.098863,0.045285,...,0.088601,0.240010,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
ENSG00000183696.9,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,7.348876,8.180940,8.721889,8.169477,...,13.428205,6.094500,12.536000,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
ENSG00000139269.2,1.199910,1.573572,0.521616,1.447225,3.565791,1.982681,0.675305,3.817395,2.561376,1.231049,...,3.225880,1.996067,2.854923,2.267343,1.331201,2.187895,1.004250,3.003316,1.984362,1.684954
ENSG00000169129.8,0.831940,0.069778,0.931086,0.620941,1.660668,0.570481,1.259393,0.734784,1.479124,1.548653,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
ENSG00000134602.11,27.646422,24.395572,16.445374,24.806650,25.113349,19.233988,27.881116,27.194117,28.579857,27.226416,...,25.079490,28.725528,24.450520,27.264069,26.912814,29.509210,26.462331,25.624009,25.707741,22.824957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235472.1,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,35.843048,40.455377,32.287466,37.636318,...,47.905182,33.224977,39.827675,27.096811,34.686342,37.386766,30.598840,33.516674,32.017940,38.341888
ENSG00000114423.14,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,15.690696,11.798242,15.461657,18.126433,...,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117
ENSG00000243312.2,1.112114,0.831797,0.253228,0.271568,0.486086,1.362640,1.543528,0.535700,0.900239,0.929809,...,1.168991,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.857330,0.703369


In [20]:
exp_data_t = exp_data_reset.transpose()
exp_data_t

Gene_Symbol,ENSG00000152931.6,ENSG00000183696.9,ENSG00000139269.2,ENSG00000169129.8,ENSG00000134602.11,ENSG00000136237.12,ENSG00000259425.1,ENSG00000242284.2,ENSG00000235027.1,ENSG00000228169.3,...,ENSG00000087095.7,ENSG00000261559.1,ENSG00000162144.4,ENSG00000129473.5,ENSG00000261205.1,ENSG00000235472.1,ENSG00000114423.14,ENSG00000243312.2,ENSG00000257337.1,ENSG00000177494.5
HG00096,0.101858,8.183805,1.199910,0.831940,27.646422,3.788503,0.054059,0.351716,0.200791,96.182178,...,5.490282,11.269426,33.050457,5.960860,0.078965,31.582832,14.054749,1.112114,3.826396,1.183398
HG00097,0.078110,5.686911,1.573572,0.069778,24.395572,2.050963,0.112185,0.444540,0.190138,101.179262,...,5.265422,11.453174,23.159734,10.834208,0.061540,34.071123,14.477899,0.831797,6.045798,0.892619
HG00099,0.048981,2.434653,0.521616,0.931086,16.445374,4.000313,0.003592,0.227708,0.092925,58.783063,...,3.457973,4.949254,16.050400,4.606371,0.098544,19.394365,11.584425,0.253228,2.593872,0.917497
HG00100,0.118597,3.830894,1.447225,0.620941,24.806650,3.271619,0.000500,0.714112,0.108790,105.483527,...,5.075550,10.291071,20.848751,5.274462,0.261279,37.523721,12.637956,0.271568,4.447169,3.504894
HG00101,0.004035,6.612288,3.565791,1.660668,25.113349,1.798216,0.029398,0.450912,0.232448,105.818192,...,5.571885,9.315269,32.245357,6.453945,0.232397,33.430473,12.015089,0.486086,5.294657,1.168247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA20815,0.088509,7.583364,2.187895,1.513521,29.509210,2.888316,0.013943,0.962397,0.121265,100.945145,...,5.643264,13.978647,26.245967,7.016099,0.198869,37.386766,19.216176,1.580658,5.333027,3.032246
NA20816,0.029204,4.052882,1.004250,0.826377,26.462331,2.145022,0.081050,0.053560,0.207399,94.643402,...,5.290116,11.694945,25.040414,6.305954,0.177514,30.598840,10.213739,0.701396,5.694370,4.558199
NA20819,0.024423,1.570378,3.003316,1.021201,25.624009,3.557598,0.070438,-0.099780,0.288634,114.259370,...,5.911672,15.264882,33.586088,6.498428,0.117468,33.516674,14.563192,0.771233,8.142939,2.807851
NA20826,0.044816,4.900372,1.984362,0.952502,25.707741,4.152063,0.049859,0.447343,0.125602,96.073384,...,5.249228,13.813213,27.436847,4.919154,0.173883,32.017940,15.637732,0.857330,5.622043,2.021329


In [21]:
# 462 rows (people), 23722 columns (genes)
exp_data_t['Description'] = 'na'

# Reorder columns to have 'description' as the first column
exp_data_desc = exp_data_t[['Description'] + [col for col in exp_data_t.columns if col != 'Description']]
exp_data_desc

Gene_Symbol,Description,ENSG00000152931.6,ENSG00000183696.9,ENSG00000139269.2,ENSG00000169129.8,ENSG00000134602.11,ENSG00000136237.12,ENSG00000259425.1,ENSG00000242284.2,ENSG00000235027.1,...,ENSG00000087095.7,ENSG00000261559.1,ENSG00000162144.4,ENSG00000129473.5,ENSG00000261205.1,ENSG00000235472.1,ENSG00000114423.14,ENSG00000243312.2,ENSG00000257337.1,ENSG00000177494.5
HG00096,na,0.101858,8.183805,1.199910,0.831940,27.646422,3.788503,0.054059,0.351716,0.200791,...,5.490282,11.269426,33.050457,5.960860,0.078965,31.582832,14.054749,1.112114,3.826396,1.183398
HG00097,na,0.078110,5.686911,1.573572,0.069778,24.395572,2.050963,0.112185,0.444540,0.190138,...,5.265422,11.453174,23.159734,10.834208,0.061540,34.071123,14.477899,0.831797,6.045798,0.892619
HG00099,na,0.048981,2.434653,0.521616,0.931086,16.445374,4.000313,0.003592,0.227708,0.092925,...,3.457973,4.949254,16.050400,4.606371,0.098544,19.394365,11.584425,0.253228,2.593872,0.917497
HG00100,na,0.118597,3.830894,1.447225,0.620941,24.806650,3.271619,0.000500,0.714112,0.108790,...,5.075550,10.291071,20.848751,5.274462,0.261279,37.523721,12.637956,0.271568,4.447169,3.504894
HG00101,na,0.004035,6.612288,3.565791,1.660668,25.113349,1.798216,0.029398,0.450912,0.232448,...,5.571885,9.315269,32.245357,6.453945,0.232397,33.430473,12.015089,0.486086,5.294657,1.168247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA20815,na,0.088509,7.583364,2.187895,1.513521,29.509210,2.888316,0.013943,0.962397,0.121265,...,5.643264,13.978647,26.245967,7.016099,0.198869,37.386766,19.216176,1.580658,5.333027,3.032246
NA20816,na,0.029204,4.052882,1.004250,0.826377,26.462331,2.145022,0.081050,0.053560,0.207399,...,5.290116,11.694945,25.040414,6.305954,0.177514,30.598840,10.213739,0.701396,5.694370,4.558199
NA20819,na,0.024423,1.570378,3.003316,1.021201,25.624009,3.557598,0.070438,-0.099780,0.288634,...,5.911672,15.264882,33.586088,6.498428,0.117468,33.516674,14.563192,0.771233,8.142939,2.807851
NA20826,na,0.044816,4.900372,1.984362,0.952502,25.707741,4.152063,0.049859,0.447343,0.125602,...,5.249228,13.813213,27.436847,4.919154,0.173883,32.017940,15.637732,0.857330,5.622043,2.021329


In [49]:
exp_data_desc.reset_index()
exp_data_desc.index.rename("NAME")
exp_data_desc

Gene_Symbol,Description,ENSG00000152931.6,ENSG00000183696.9,ENSG00000139269.2,ENSG00000169129.8,ENSG00000134602.11,ENSG00000136237.12,ENSG00000259425.1,ENSG00000242284.2,ENSG00000235027.1,...,ENSG00000087095.7,ENSG00000261559.1,ENSG00000162144.4,ENSG00000129473.5,ENSG00000261205.1,ENSG00000235472.1,ENSG00000114423.14,ENSG00000243312.2,ENSG00000257337.1,ENSG00000177494.5
HG00096,na,0.101858,8.183805,1.199910,0.831940,27.646422,3.788503,0.054059,0.351716,0.200791,...,5.490282,11.269426,33.050457,5.960860,0.078965,31.582832,14.054749,1.112114,3.826396,1.183398
HG00097,na,0.078110,5.686911,1.573572,0.069778,24.395572,2.050963,0.112185,0.444540,0.190138,...,5.265422,11.453174,23.159734,10.834208,0.061540,34.071123,14.477899,0.831797,6.045798,0.892619
HG00099,na,0.048981,2.434653,0.521616,0.931086,16.445374,4.000313,0.003592,0.227708,0.092925,...,3.457973,4.949254,16.050400,4.606371,0.098544,19.394365,11.584425,0.253228,2.593872,0.917497
HG00100,na,0.118597,3.830894,1.447225,0.620941,24.806650,3.271619,0.000500,0.714112,0.108790,...,5.075550,10.291071,20.848751,5.274462,0.261279,37.523721,12.637956,0.271568,4.447169,3.504894
HG00101,na,0.004035,6.612288,3.565791,1.660668,25.113349,1.798216,0.029398,0.450912,0.232448,...,5.571885,9.315269,32.245357,6.453945,0.232397,33.430473,12.015089,0.486086,5.294657,1.168247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA20815,na,0.088509,7.583364,2.187895,1.513521,29.509210,2.888316,0.013943,0.962397,0.121265,...,5.643264,13.978647,26.245967,7.016099,0.198869,37.386766,19.216176,1.580658,5.333027,3.032246
NA20816,na,0.029204,4.052882,1.004250,0.826377,26.462331,2.145022,0.081050,0.053560,0.207399,...,5.290116,11.694945,25.040414,6.305954,0.177514,30.598840,10.213739,0.701396,5.694370,4.558199
NA20819,na,0.024423,1.570378,3.003316,1.021201,25.624009,3.557598,0.070438,-0.099780,0.288634,...,5.911672,15.264882,33.586088,6.498428,0.117468,33.516674,14.563192,0.771233,8.142939,2.807851
NA20826,na,0.044816,4.900372,1.984362,0.952502,25.707741,4.152063,0.049859,0.447343,0.125602,...,5.249228,13.813213,27.436847,4.919154,0.173883,32.017940,15.637732,0.857330,5.622043,2.021329


In [24]:
exp_data_desc.to_csv('gene_exp_format.txt', sep='\t')

In [38]:
s = "HG00096	HG00097	HG00099	HG00100	HG00101	HG00102	HG00103	HG00104	HG00105	HG00106	HG00108	HG00109	HG00110	HG00111	HG00112	HG00114	HG00115	HG00116	HG00117	HG00118	HG00119	HG00120	HG00121	HG00122	HG00123	HG00124	HG00125	HG00126	HG00127	HG00128	HG00129	HG00130	HG00131	HG00132	HG00133	HG00134	HG00135	HG00136	HG00137	HG00138	HG00139	HG00141	HG00142	HG00143	HG00145	HG00146	HG00148	HG00149	HG00150	HG00151	HG00152	HG00154	HG00155	HG00156	HG00157	HG00158	HG00159	HG00160	HG00171	HG00173	HG00174	HG00176	HG00177	HG00178	HG00179	HG00180	HG00181	HG00182	HG00183	HG00185	HG00186	HG00187	HG00188	HG00189	HG00231	HG00232	HG00233	HG00234	HG00235	HG00236	HG00238	HG00239	HG00240	HG00242	HG00243	HG00244	HG00245	HG00246	HG00247	HG00249	HG00250	HG00251	HG00252	HG00253	HG00255	HG00256	HG00257	HG00258	HG00259	HG00260	HG00261	HG00262	HG00263	HG00264	HG00265	HG00266	HG00267	HG00268	HG00269	HG00271	HG00272	HG00273	HG00274	HG00275	HG00276	HG00277	HG00278	HG00280	HG00281	HG00282	HG00284	HG00285	HG00306	HG00308	HG00309	HG00310	HG00311	HG00312	HG00313	HG00315	HG00319	HG00320	HG00321	HG00323	HG00324	HG00325	HG00326	HG00327	HG00328	HG00329	HG00330	HG00331	HG00332	HG00334	HG00335	HG00336	HG00337	HG00338	HG00339	HG00341	HG00342	HG00343	HG00344	HG00345	HG00346	HG00349	HG00350	HG00351	HG00353	HG00355	HG00356	HG00358	HG00359	HG00360	HG00361	HG00362	HG00364	HG00365	HG00366	HG00367	HG00369	HG00371	HG00372	HG00373	HG00375	HG00376	HG00377	HG00378	HG00379	HG00380	HG00381	HG00382	HG00383	HG00384	HG01334	HG01789	HG01790	HG01791	HG02215	NA06984	NA06985	NA06986	NA06989	NA06994	NA07037	NA07048	NA07051	NA07056	NA07346	NA07347	NA07357	NA10847	NA10851	NA11829	NA11830	NA11831	NA11832	NA11840	NA11843	NA11881	NA11892	NA11893	NA11894	NA11918	NA11920	NA11930	NA11931	NA11992	NA11993	NA11994	NA11995	NA12004	NA12005	NA12006	NA12043	NA12044	NA12045	NA12058	NA12144	NA12154	NA12155	NA12156	NA12234	NA12249	NA12272	NA12273	NA12275	NA12282	NA12283	NA12286	NA12287	NA12340	NA12341	NA12342	NA12347	NA12348	NA12383	NA12399	NA12400	NA12413	NA12489	NA12546	NA12716	NA12717	NA12718	NA12749	NA12750	NA12751	NA12760	NA12761	NA12762	NA12763	NA12775	NA12776	NA12777	NA12778	NA12812	NA12813	NA12814	NA12815	NA12827	NA12829	NA12830	NA12842	NA12843	NA12872	NA12873	NA12874	NA12889	NA12890	NA20502	NA20503	NA20504	NA20505	NA20506	NA20507	NA20508	NA20509	NA20510	NA20512	NA20513	NA20514	NA20515	NA20516	NA20517	NA20518	NA20519	NA20520	NA20521	NA20524	NA20525	NA20527	NA20528	NA20529	NA20530	NA20531	NA20532	NA20534	NA20535	NA20536	NA20537	NA20538	NA20539	NA20540	NA20541	NA20542	NA20543	NA20544	NA20581	NA20582	NA20585	NA20586	NA20588	NA20589	NA20752	NA20754	NA20756	NA20757	NA20758	NA20759	NA20760	NA20761	NA20765	NA20766	NA20768	NA20769	NA20770	NA20771	NA20772	NA20773	NA20774	NA20778	NA20783	NA20785	NA20786	NA20787	NA20790	NA20792	NA20795	NA20796	NA20797	NA20798	NA20799	NA20800	NA20801	NA20802	NA20803	NA20804	NA20805	NA20806	NA20807	NA20808	NA20809	NA20810	NA20811	NA20812	NA20813	NA20814	NA20815	NA20816	NA20819	NA20826	NA20828"
arr = s.split("	")
len(arr)

373

In [50]:
df_exp_common = exp_data_desc.reindex(arr, fill_value=0)
df_exp_common

Gene_Symbol,Description,ENSG00000152931.6,ENSG00000183696.9,ENSG00000139269.2,ENSG00000169129.8,ENSG00000134602.11,ENSG00000136237.12,ENSG00000259425.1,ENSG00000242284.2,ENSG00000235027.1,...,ENSG00000087095.7,ENSG00000261559.1,ENSG00000162144.4,ENSG00000129473.5,ENSG00000261205.1,ENSG00000235472.1,ENSG00000114423.14,ENSG00000243312.2,ENSG00000257337.1,ENSG00000177494.5
HG00096,na,0.101858,8.183805,1.199910,0.831940,27.646422,3.788503,0.054059,0.351716,0.200791,...,5.490282,11.269426,33.050457,5.960860,0.078965,31.582832,14.054749,1.112114,3.826396,1.183398
HG00097,na,0.078110,5.686911,1.573572,0.069778,24.395572,2.050963,0.112185,0.444540,0.190138,...,5.265422,11.453174,23.159734,10.834208,0.061540,34.071123,14.477899,0.831797,6.045798,0.892619
HG00099,na,0.048981,2.434653,0.521616,0.931086,16.445374,4.000313,0.003592,0.227708,0.092925,...,3.457973,4.949254,16.050400,4.606371,0.098544,19.394365,11.584425,0.253228,2.593872,0.917497
HG00100,na,0.118597,3.830894,1.447225,0.620941,24.806650,3.271619,0.000500,0.714112,0.108790,...,5.075550,10.291071,20.848751,5.274462,0.261279,37.523721,12.637956,0.271568,4.447169,3.504894
HG00101,na,0.004035,6.612288,3.565791,1.660668,25.113349,1.798216,0.029398,0.450912,0.232448,...,5.571885,9.315269,32.245357,6.453945,0.232397,33.430473,12.015089,0.486086,5.294657,1.168247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA20815,na,0.088509,7.583364,2.187895,1.513521,29.509210,2.888316,0.013943,0.962397,0.121265,...,5.643264,13.978647,26.245967,7.016099,0.198869,37.386766,19.216176,1.580658,5.333027,3.032246
NA20816,na,0.029204,4.052882,1.004250,0.826377,26.462331,2.145022,0.081050,0.053560,0.207399,...,5.290116,11.694945,25.040414,6.305954,0.177514,30.598840,10.213739,0.701396,5.694370,4.558199
NA20819,na,0.024423,1.570378,3.003316,1.021201,25.624009,3.557598,0.070438,-0.099780,0.288634,...,5.911672,15.264882,33.586088,6.498428,0.117468,33.516674,14.563192,0.771233,8.142939,2.807851
NA20826,na,0.044816,4.900372,1.984362,0.952502,25.707741,4.152063,0.049859,0.447343,0.125602,...,5.249228,13.813213,27.436847,4.919154,0.173883,32.017940,15.637732,0.857330,5.622043,2.021329


In [45]:
df_exp_common.to_csv('gene_exp_format_common.txt', sep='\t')

In [44]:
s2 = "1	2	2	2	1	2	1	2	1	2	1	1	2	2	1	1	1	1	1	2	1	2	2	2	2	2	2	1	2	2	1	2	1	2	2	2	2	1	2	1	1	1	1	1	1	2	1	1	2	1	1	2	1	1	1	2	1	1	2	2	2	2	2	2	2	2	1	1	1	1	1	1	1	1	2	2	2	1	2	2	2	2	2	1	1	1	2	1	2	2	2	1	1	2	2	1	2	2	2	1	2	2	2	1	1	2	1	2	2	1	2	1	2	2	2	1	1	1	2	2	1	2	2	1	2	1	1	1	2	2	2	2	1	2	2	1	2	2	2	1	2	2	2	2	1	1	2	1	2	1	1	2	2	1	2	2	2	1	2	2	2	1	2	1	2	2	2	2	1	2	1	1	1	2	1	2	2	2	2	2	2	1	2	2	1	1	2	1	2	1	2	1	2	1	2	1	1	2	2	1	1	2	1	1	2	1	2	2	1	1	2	1	2	2	2	1	2	1	2	1	2	2	1	2	1	2	1	2	1	1	1	2	2	2	1	2	2	1	2	1	2	1	2	1	1	2	2	1	2	1	2	1	1	2	2	2	1	2	1	2	1	2	1	2	1	2	1	2	1	2	1	1	2	1	2	1	2	1	1	2	2	2	2	2	2	2	2	1	1	1	1	2	1	1	2	1	1	1	1	1	1	1	1	2	2	2	1	1	2	1	1	1	1	2	2	2	1	1	1	2	2	1	1	2	1	1	2	2	1	1	2	2	1	2	2	2	1	2	2	2	2	1	1	1	2	1	2	1	2	1	2	1	2	2	1	2	1	2	1	1	2	2	1	1	1	1	2	1	1	1	2	2	2"
arr2 = s2.split(	)
c1 = arr2.count("1")
c2 = arr2.count("2")
c1, c2

(176, 197)

In [84]:
exp_data_format = df_exp_common.transpose()
exp_data_format = exp_data_format.drop(exp_data_format.index[0])
# 462 rows (people), 23722 columns (genes)
exp_data_format['Description'] = 'na'

# Reorder columns to have 'description' as the first column
exp_data_form = exp_data_format[['Description'] + [col for col in exp_data_format.columns if col != 'Description']]
exp_data_form

,Description,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,HG00105,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
Gene_Symbol,,,,,,,,,,,,,,,,,,,,,
ENSG00000152931.6,na,0.101858,0.07811,0.048981,0.118597,0.004035,0.010925,-0.000901,-0.006706,0.098863,...,0.088601,0.24001,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
ENSG00000183696.9,na,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,7.348876,8.18094,8.721889,...,13.428205,6.0945,12.536,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
ENSG00000139269.2,na,1.19991,1.573572,0.521616,1.447225,3.565791,1.982681,0.675305,3.817395,2.561376,...,3.22588,1.996067,2.854923,2.267343,1.331201,2.187895,1.00425,3.003316,1.984362,1.684954
ENSG00000169129.8,na,0.83194,0.069778,0.931086,0.620941,1.660668,0.570481,1.259393,0.734784,1.479124,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
ENSG00000134602.11,na,27.646422,24.395572,16.445374,24.80665,25.113349,19.233988,27.881116,27.194117,28.579857,...,25.07949,28.725528,24.45052,27.264069,26.912814,29.50921,26.462331,25.624009,25.707741,22.824957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235472.1,na,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,35.843048,40.455377,32.287466,...,47.905182,33.224977,39.827675,27.096811,34.686342,37.386766,30.59884,33.516674,32.01794,38.341888
ENSG00000114423.14,na,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,15.690696,11.798242,15.461657,...,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117
ENSG00000243312.2,na,1.112114,0.831797,0.253228,0.271568,0.486086,1.36264,1.543528,0.5357,0.900239,...,1.168991,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.85733,0.703369


In [85]:
def remove_dot(s):
    return s.split(".")[0]
exp_data_form = exp_data_form.reset_index()
exp_data_form['Gene_Symbol'] = exp_data_form['Gene_Symbol'].apply(remove_dot)
exp_data_form = exp_data_form.set_index(['Gene_Symbol'])
exp_data_form

,Description,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,HG00105,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
Gene_Symbol,,,,,,,,,,,,,,,,,,,,,
ENSG00000152931,na,0.101858,0.07811,0.048981,0.118597,0.004035,0.010925,-0.000901,-0.006706,0.098863,...,0.088601,0.24001,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
ENSG00000183696,na,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,7.348876,8.18094,8.721889,...,13.428205,6.0945,12.536,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
ENSG00000139269,na,1.19991,1.573572,0.521616,1.447225,3.565791,1.982681,0.675305,3.817395,2.561376,...,3.22588,1.996067,2.854923,2.267343,1.331201,2.187895,1.00425,3.003316,1.984362,1.684954
ENSG00000169129,na,0.83194,0.069778,0.931086,0.620941,1.660668,0.570481,1.259393,0.734784,1.479124,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
ENSG00000134602,na,27.646422,24.395572,16.445374,24.80665,25.113349,19.233988,27.881116,27.194117,28.579857,...,25.07949,28.725528,24.45052,27.264069,26.912814,29.50921,26.462331,25.624009,25.707741,22.824957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235472,na,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,35.843048,40.455377,32.287466,...,47.905182,33.224977,39.827675,27.096811,34.686342,37.386766,30.59884,33.516674,32.01794,38.341888
ENSG00000114423,na,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,15.690696,11.798242,15.461657,...,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117
ENSG00000243312,na,1.112114,0.831797,0.253228,0.271568,0.486086,1.36264,1.543528,0.5357,0.900239,...,1.168991,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.85733,0.703369


In [86]:
exp_data_form.to_csv('gene_exp_format_common.txt', sep='\t')

In [87]:
gene_annot = pd.read_csv('gene_annot.txt', sep='\t')
gene_annot = gene_annot.set_index('SYM')
gene_annot

,ID,CHR,START,STOP,TYPE
SYM,,,,,
ENSG00000223972,DDX11L1,1,11868,13052,transcribed_unprocessed_pseudogene
ENSG00000186092,OR4F5,1,69090,70008,protein_coding
ENSG00000177757,FAM87B,1,817370,819834,lincRNA
ENSG00000225880,LINC00115,1,826205,827522,lincRNA
ENSG00000228794,LINC01128,1,827607,859446,processed_transcript
...,...,...,...,...,...
ENSG00000100299,ARSA,22,50622753,50628179,protein_coding
ENSG00000251322,SHANK3,22,50674414,50733298,protein_coding
ENSG00000100312,ACR,22,50738195,50745334,protein_coding


In [88]:
gene_map = gene_annot[['ID']]
gene_map

,ID
SYM,
ENSG00000223972,DDX11L1
ENSG00000186092,OR4F5
ENSG00000177757,FAM87B
ENSG00000225880,LINC00115
ENSG00000228794,LINC01128
...,...
ENSG00000100299,ARSA
ENSG00000251322,SHANK3
ENSG00000100312,ACR


In [89]:
merged_df = pd.merge(exp_data_form, gene_map, left_index=True, right_index=True, how='left')
merged_df

,Description,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,HG00105,...,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828,ID
Gene_Symbol,,,,,,,,,,,,,,,,,,,,,
ENSG00000152931,na,0.101858,0.07811,0.048981,0.118597,0.004035,0.010925,-0.000901,-0.006706,0.098863,...,0.24001,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186,PART1
ENSG00000183696,na,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,7.348876,8.18094,8.721889,...,6.0945,12.536,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308,UPP1
ENSG00000139269,na,1.19991,1.573572,0.521616,1.447225,3.565791,1.982681,0.675305,3.817395,2.561376,...,1.996067,2.854923,2.267343,1.331201,2.187895,1.00425,3.003316,1.984362,1.684954,INHBE
ENSG00000169129,na,0.83194,0.069778,0.931086,0.620941,1.660668,0.570481,1.259393,0.734784,1.479124,...,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565,AFAP1L2
ENSG00000134602,na,27.646422,24.395572,16.445374,24.80665,25.113349,19.233988,27.881116,27.194117,28.579857,...,28.725528,24.45052,27.264069,26.912814,29.50921,26.462331,25.624009,25.707741,22.824957,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235472,na,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,35.843048,40.455377,32.287466,...,33.224977,39.827675,27.096811,34.686342,37.386766,30.59884,33.516674,32.01794,38.341888,NaN
ENSG00000114423,na,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,15.690696,11.798242,15.461657,...,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117,CBLB
ENSG00000243312,na,1.112114,0.831797,0.253228,0.271568,0.486086,1.36264,1.543528,0.5357,0.900239,...,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.85733,0.703369,NaN


In [90]:
nan_df = merged_df[merged_df['ID'].isna()]
nan_df

,Description,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,HG00105,...,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828,ID
Gene_Symbol,,,,,,,,,,,,,,,,,,,,,
ENSG00000134602,na,27.646422,24.395572,16.445374,24.80665,25.113349,19.233988,27.881116,27.194117,28.579857,...,28.725528,24.45052,27.264069,26.912814,29.50921,26.462331,25.624009,25.707741,22.824957,NaN
ENSG00000259425,na,0.054059,0.112185,0.003592,0.0005,0.029398,0.031266,0.105538,0.058953,0.062852,...,0.010224,0.000204,0.059104,0.066048,0.013943,0.08105,0.070438,0.049859,0.017376,NaN
ENSG00000242284,na,0.351716,0.44454,0.227708,0.714112,0.450912,0.491438,2.954486,0.726429,0.033625,...,1.682329,0.68678,1.20754,0.088764,0.962397,0.05356,-0.09978,0.447343,0.002862,NaN
ENSG00000235027,na,0.200791,0.190138,0.092925,0.10879,0.232448,0.250905,0.104912,0.134485,0.264147,...,0.254004,0.294359,0.172155,0.135213,0.121265,0.207399,0.288634,0.125602,0.141499,NaN
ENSG00000228169,na,96.182178,101.179262,58.783063,105.483527,105.818192,136.140843,70.680737,103.441921,90.962547,...,103.412712,81.928224,106.792474,100.898778,100.945145,94.643402,114.25937,96.073384,105.326429,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000261559,na,11.269426,11.453174,4.949254,10.291071,9.315269,17.223121,10.681895,9.866422,14.239237,...,10.179616,16.12838,10.889905,15.059398,13.978647,11.694945,15.264882,13.813213,15.198913,NaN
ENSG00000261205,na,0.078965,0.06154,0.098544,0.261279,0.232397,0.289815,0.189539,0.147931,0.310231,...,0.218323,0.083362,0.254885,0.330083,0.198869,0.177514,0.117468,0.173883,0.18796,NaN
ENSG00000235472,na,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,35.843048,40.455377,32.287466,...,33.224977,39.827675,27.096811,34.686342,37.386766,30.59884,33.516674,32.01794,38.341888,NaN


In [98]:
no_nan = merged_df[merged_df['ID'].notna()]
no_nan = no_nan.set_index('ID')
no_nan

,Description,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,HG00105,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
ID,,,,,,,,,,,,,,,,,,,,,
PART1,na,0.101858,0.07811,0.048981,0.118597,0.004035,0.010925,-0.000901,-0.006706,0.098863,...,0.088601,0.24001,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
UPP1,na,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,7.348876,8.18094,8.721889,...,13.428205,6.0945,12.536,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
INHBE,na,1.19991,1.573572,0.521616,1.447225,3.565791,1.982681,0.675305,3.817395,2.561376,...,3.22588,1.996067,2.854923,2.267343,1.331201,2.187895,1.00425,3.003316,1.984362,1.684954
AFAP1L2,na,0.83194,0.069778,0.931086,0.620941,1.660668,0.570481,1.259393,0.734784,1.479124,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
RAPGEF5,na,3.788503,2.050963,4.000313,3.271619,1.798216,1.516688,2.500333,4.509277,2.344625,...,2.909393,1.921176,5.083873,2.866573,1.297788,2.888316,2.145022,3.557598,4.152063,1.216834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NLK,na,5.490282,5.265422,3.457973,5.07555,5.571885,5.262668,5.371397,4.001221,6.861067,...,6.678943,4.996638,5.125125,5.38347,5.206981,5.643264,5.290116,5.911672,5.249228,5.347071
CYB561A3,na,33.050457,23.159734,16.0504,20.848751,32.245357,26.842346,34.050713,23.646787,30.473699,...,22.962417,26.718492,17.096197,36.692807,32.645718,26.245967,25.040414,33.586088,27.436847,24.23684
BCL2L2,na,5.96086,10.834208,4.606371,5.274462,6.453945,11.956208,4.315196,7.256263,5.556447,...,7.947619,5.826466,6.691473,11.671064,9.606188,7.016099,6.305954,6.498428,4.919154,7.285334


In [99]:
no_nan.to_csv('gene_exp_format_common.txt', sep='\t')